# Data Processing

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

IMAGE_DIR = 'figures/'  # output images directory


def load_dataset(dataset='MADELON', split_percentage=0.2):

    datasets = ['MADELON', 'MNIST']  # datasets names
    print('\nLoading {} Dataset'.format(dataset))

    if dataset == datasets[0]:

        data = fetch_openml('madelon')
        x, y = data.data, data.target
        x, _, y, _ = train_test_split(x, y, test_size=None, shuffle=True, random_state=42, stratify=y)
        labels = ['0', '1']  # labels
        y = y.astype(int)

    elif dataset == datasets[1]:

        data = fetch_openml('mnist_784')
        x, y = data.data, data.target
        x, _, y, _ = train_test_split(x, y, test_size=0.90, shuffle=True, random_state=42, stratify=y)
        labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  # labels
        y = y.astype(int)

    # Split dataset in training and validation sets, preserving classes representation
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=split_percentage, shuffle=True,
                                                        random_state=42, stratify=y)

    # Normalize feature data
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    print('\nTotal dataset size:')
    print('Number of instances: {}'.format(x.shape[0]))
    print('Number of features: {}'.format(x.shape[1]))
    print('Number of classes: {}'.format(len(labels)))
    print('Training Set : {}'.format(x_train.shape))
    print('Testing Set : {}'.format(x_test.shape))

    return x_train, x_test, y_train, y_test

# Clustering

In [2]:
# Clustering script

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import utils

from abc import ABC, abstractmethod

from sklearn.base import clone
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, completeness_score, \
                            homogeneity_score, v_measure_score, silhouette_samples, silhouette_score
from sklearn.mixture import GaussianMixture


class Clustering(ABC):

    def __init__(self, name, n_clusters, max_n_clusters, name_param, random_seed):
       
        self.name = name
        self.model = None
        self.clusters = None
        self.n_clusters = n_clusters
        self.max_n_clusters = max_n_clusters
        self.name_param = name_param
        self.random_seed = random_seed

    @staticmethod
    def benchmark(x, y, clusters):
       
        print('homo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
        print('{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}'.format(homogeneity_score(y, clusters),
                                                                      completeness_score(y, clusters),
                                                                      v_measure_score(y, clusters),
                                                                      adjusted_rand_score(y, clusters),
                                                                      adjusted_mutual_info_score(y, clusters, average_method='arithmetic'),
                                                                      silhouette_score(x, clusters, metric='euclidean')))

    def experiment(self, x_train, x_test, y_train, dataset, perform_model_complexity):
        
        if perform_model_complexity:
            self.plot_model_complexity(x_train, y_train, dataset)

        self.train(x_train, y_train)  # fit the model and benchmark on training data
        self.visualize_clusters(x_train, y_train, dataset)  # visualize clusters with PCA and TSNE

        return self.clusters, self.predict(x_test)  # predict on test data and return training and test clusters

    @abstractmethod
    def plot_model_complexity(self, x, dataset):
       
        pass

    def predict(self, x):

        return self.model.predict(x)

    def set_n_clusters(self, n_clusters):
       
        self.n_clusters = n_clusters
        self.model.set_params(**{'n_clusters': n_clusters})

    def train(self, x, y):
       
        print('\nTrain on training set with k={}'.format(self.n_clusters))
        self.clusters = self.model.fit_predict(x)
        self.benchmark(x, y, self.clusters)

    def visualize_clusters(self, x, y, dataset):
       

        # Declare PCA and reduce data
        pca = PCA(n_components=2, random_state=self.random_seed)
        x_pca = pca.fit_transform(x)

        # Declare TSNE and reduce data
        tsne = TSNE(n_components=2, random_state=self.random_seed)
        x_tsne = tsne.fit_transform(x)

        n_classes = len(np.unique(y))  # compute number of classes
        print('\nBenchmark Model with k = n classes = {}'.format(n_classes))

        # Benchamark the model with number of clusters (k) = number of classes
        model = clone(self.model)
        model_params = self.model.get_params()
        model_params[self.name_param] = n_classes
        model.set_params(**model_params)
        clusters = model.fit_predict(x)
        self.benchmark(x, y, clusters)

        # Create dataframe for visualization
        df = pd.DataFrame(x_tsne, columns=['tsne1', 'tsne2'])
        df['pca1'] = x_pca[:, 0]
        df['pca2'] = x_pca[:, 1]
        df['y'] = y
        df['c'] = self.clusters

        # Create subplot and plot clusters with PCA and TSNE
        fig, (ax1, ax2) = plt.subplots(2, 2, figsize=(15, 8))
        utils.plot_clusters(ax1, 'pca1', 'pca2', df, self.name)
        utils.plot_clusters(ax2, 'tsne1', 'tsne2', df, self.name)

        # Save figure
        utils.save_figure_tight('{}_{}_clusters'.format(dataset, self.name))

## K-Means

In [3]:
class KMeansClustering(Clustering):

    def __init__(self,  n_clusters=2, max_n_clusters=10, random_seed=42):
    
        super(KMeansClustering, self).__init__(name='k-means', n_clusters=n_clusters, max_n_clusters=max_n_clusters,
                                               name_param='n_clusters', random_seed=random_seed)
        self.model = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=1000, random_state=random_seed, n_jobs=-1)

    def plot_model_complexity(self, x, y, dataset):
      
        print('\nPlot Model Complexity')

        inertia, inertia_diff = [], []  # inertia and delta inertia
        if "MADELON" in dataset:
            k_range = np.arange(2, self.max_n_clusters + 1)  # range of number of clusters k to plot over
        elif "MNIST" in dataset:
            k_range = [1, 2, 5, 8, 10, 12, 15, 20]
        homogeneity = []
        completeness = []
        silhouette = []
        # For each k in the range
        for k in k_range:

            # Define a new k-Means model, fit on training data and report inertia
            model = KMeans(n_clusters=k, init='k-means++', max_iter=1000, random_state=self.random_seed, n_jobs=-1)
            clusters = model.fit_predict(x)
            inertia.append(model.inertia_)
            h_score = homogeneity_score(y, clusters)
            if k != 1:
                s_score = silhouette_score(x, clusters, metric='euclidean')
            else:
                s_score = 0
            c_score = completeness_score(y, clusters)
            homogeneity.append(h_score)
            silhouette.append(s_score)
            completeness.append(c_score)
            print('k = {} -->  inertia = {:.3f}  silhouette = {:.3f} homogeneity = {:.3f} completeness = {:.3f}'.format(k,
                                                                                                 inertia[-1],
                                                                                                 h_score,
                                                                                                 s_score,
                                                                                                 c_score,
                                                                                                 ))


        # Create subplots and plot inertia and delta inertia on the first suplot
        fig, ax = plt.subplots(1, 4, figsize=(16, 4))
        ax = ax.flatten()
        ax[0].plot(k_range, inertia, '-o', markersize=2, label='Inertia')

        # Set legend, title and labels
        utils.set_axis_title_labels(ax[0], title='K-Means - Inertia',
                                    x_label='Number of clusters k', y_label='Inertia')

        ax[1].plot(k_range[1:], silhouette[1:], '-o', markersize=2, label='Silhouette')
        utils.set_axis_title_labels(ax[2], title='K-Means - Silhouette Score',
                                    x_label='Number of clusters k', y_label='Silhouette')
        ax[2].plot(k_range, homogeneity, '-o', markersize=2, label='Homogeneity')
        utils.set_axis_title_labels(ax[1], title='K-Means - Homogeneity Score',
                                    x_label='Number of clusters k', y_label='Homogeneity')
        ax[3].plot(k_range, completeness, '-o', markersize=2, label='Completeness')
        utils.set_axis_title_labels(ax[3], title='K-Means - Completeness Score',
                                    x_label='Number of clusters k', y_label='Completeness')
        # Save figure
        utils.save_figure_tight('{}_{}_model_complexity'.format(dataset, self.name))

## Expectation Maximization

In [4]:
class MixtureOfGaussians(Clustering):
    
    def __init__(self, n_clusters=2, covariance='full', max_n_clusters=10, random_seed=42):
        
        super(MixtureOfGaussians, self).__init__(name='em', n_clusters=n_clusters, max_n_clusters=max_n_clusters,
                                                 name_param='n_components', random_seed=random_seed)
        self.model = GaussianMixture(n_components=n_clusters, covariance_type=covariance, max_iter=1000,
                                     n_init=10, init_params='random', random_state=random_seed)

    def plot_model_complexity(self, x, y, dataset):
        

        print('\nPlot Model Complexity')
        if "MADELON" in dataset:
            k_range = np.arange(2, self.max_n_clusters + 1)  # range of number of clusters k to plot over
        elif "MNIST" in dataset:
            k_range = [1, 2, 5, 8, 10, 12, 15, 20]
        # cv_types = ['spherical', 'tied', 'diag', 'full']  # covariance types to plot over
        cv_types = ['diag']  # covariance types to plot over
        fig, ax = plt.subplots(1, 4, figsize=(16, 4))
        homogeneity = []
        completeness = []
        # For all pair covariance, number of clusters k
        for cv_type in cv_types:
            aic, bic = [], []  # AIC and BIC  scores lists
            for k in k_range:

                # Define a new Gaussian Mixture model, fit on training data and report AIC and BIC
                gmm = GaussianMixture(n_components=k, covariance_type=cv_type, max_iter=1000,
                                      n_init=10, init_params='random', random_state=self.random_seed)
                clusters = gmm.fit_predict(x)
                aic.append(gmm.aic(x))
                bic.append(gmm.bic(x))

                print('cv = {}, k = {} --> aic = {:.3f}, bic = {:.3f}'.format(cv_type, k, aic[-1], bic[-1]))
                if cv_type == 'diag':
                    h_score = homogeneity_score(y, clusters)
                    c_score = completeness_score(y, clusters)
                    homogeneity.append(h_score)
                    completeness.append(c_score)
                    print('cv_type = {}, k = {} --> homogeneity = {:.3f}, completeness = {:.3f}'.format(cv_type, k,
                                                                                                        h_score,
                                                                                                        c_score))

            ax[0].plot(k_range, aic, '-o', label=cv_type)
            ax[1].plot(k_range, bic, '-o', label=cv_type)

        for i in range(4):
            ax[i].set_xticks(k_range)

        ax[0].legend(loc='best')
        ax[1].legend(loc='best')

        utils.set_axis_title_labels(ax[0], title='EM - AIC Score',
                                    x_label='Number of clusters k', y_label='AIC')
        utils.set_axis_title_labels(ax[1], title='EM - BIC Score',
                                    x_label='Number of clusters k', y_label='BIC')

        ax[2].plot(k_range, homogeneity, '-o', markersize=2, label='Homogeneity')
        utils.set_axis_title_labels(ax[2], title='EM - Homogeneity Score',
                                    x_label='Number of clusters k', y_label='Homogeneity')
        ax[3].plot(k_range, completeness, '-o', markersize=2, label='Completeness')
        utils.set_axis_title_labels(ax[3], title='EM - Completeness Score',
                                    x_label='Number of clusters k', y_label='Completeness')
        # Save figure
        utils.save_figure_tight('{}_{}_model_complexity'.format(dataset, self.name))

## Running Clustering

In [5]:
def clustering(x_train, x_test, y_train, **kwargs):
    print('\n--------------------------')
    print('kMeans')

    # # Declare kMeans, perform experiments and get clusters on training data
    kmeans = KMeansClustering(n_clusters=kwargs['kmeans_n_clusters'], max_n_clusters=10)
    kmeans_clusters = kmeans.experiment(x_train, x_test, y_train,
                                        dataset=kwargs['dataset'],
                                        perform_model_complexity=kwargs['perform_model_complexity'])

    print('\n--------------------------')
    print('GMM')

    # Declare Gaussian Mixtures Models, perform experiments and get clusters on training data
    gmm = MixtureOfGaussians(n_clusters=kwargs['em_n_clusters'], covariance=kwargs['em_covariance'], max_n_clusters=10)
    gmm_clusters = gmm.experiment(x_train, x_test, y_train,
                                  dataset=kwargs['dataset'],
                                  perform_model_complexity=kwargs['perform_model_complexity'])

    return kmeans_clusters, gmm_clusters


In [7]:
dataset = 'MNIST'
x_train, x_test, y_train, y_test = load_dataset(dataset)
kmeans_clusters, gmm_clusters = clustering(x_train, x_test, y_train,
                                                   dataset=dataset,
                                                   kmeans_n_clusters=6,
                                                   em_n_clusters=10, em_covariance='full',
                                                   perform_model_complexity=True)


Loading MNIST Dataset

Total dataset size:
Number of instances: 7000
Number of features: 784
Number of classes: 10
Training Set : (5600, 784)
Testing Set : (1400, 784)

--------------------------
kMeans

Plot Model Complexity


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 1 -->  inertia = 3724000.000  silhouette = 0.000 homogeneity = 0.000 completeness = 1.000


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 2 -->  inertia = 3571445.749  silhouette = 0.085 homogeneity = 0.128 completeness = 0.323


...

k = 20 -->  inertia = 2853873.608  silhouette = 0.530 homogeneity = -0.008 completeness = 0.442

Train on training set with k=6


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


homo	compl	v-meas	ARI	AMI	silhouette
0.339	0.455	0.389	0.269	0.387	-0.003

Benchmark Model with k = n classes = 10


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


homo	compl	v-meas	ARI	AMI	silhouette
0.421	0.446	0.433	0.319	0.432	0.005

--------------------------
GMM

Plot Model Complexity
cv = diag, k = 1 --> aic = 2589495.237, bic = 2599891.895
cv_type = diag, k = 1 --> homogeneity = 0.000, completeness = 1.000
cv = diag, k = 2 --> aic = -5470985.602, bic = -5450185.655
cv_type = diag, k = 2 --> homogeneity = 0.016, completeness = 0.094
cv = diag, k = 5 --> aic = -12918684.505, bic = -12866674.691
cv_type = diag, k = 5 --> homogeneity = 0.077, completeness = 0.153
cv = diag, k = 8 --> aic = -15374107.457, bic = -15290887.777
cv_type = diag, k = 8 --> homogeneity = 0.172, completeness = 0.243
cv = diag, k = 10 --> aic = -16130289.523, bic = -16026263.266
cv_type = diag, k = 10 --> homogeneity = 0.200, completeness = 0.236
cv = diag, k = 12 --> aic = -18364011.512, bic = -18239178.677
cv_type = diag, k = 12 --> homogeneity = 0.183, completeness = 0.220
cv = diag, k = 15 --> aic = -18361012.956, bic = -18204970.255
cv_type = diag, k = 15 --> homo

In [6]:
dataset = 'MADELON'
x_train, x_test, y_train, y_test = load_dataset(dataset)
kmeans_clusters, gmm_clusters = clustering(x_train, x_test, y_train,
                                                   dataset=dataset,
                                                   kmeans_n_clusters=6,
                                                   em_n_clusters=10, em_covariance='full',
                                                   perform_model_complexity=True)


Loading MADELON Dataset

Total dataset size:
Number of instances: 1950
Number of features: 500
Number of classes: 2
Training Set : (1560, 500)
Testing Set : (390, 500)

--------------------------
kMeans

Plot Model Complexity


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 2 -->  inertia = 772874.876  silhouette = 0.015 homogeneity = 0.008 completeness = 0.015


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 3 -->  inertia = 768996.931  silhouette = 0.033 homogeneity = 0.006 completeness = 0.021


...

k = 10 -->  inertia = 759005.094  silhouette = 0.016 homogeneity = 0.003 completeness = 0.005

Train on training set with k=6


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


homo	compl	v-meas	ARI	AMI	silhouette
0.031	0.012	0.017	0.012	0.016	0.005

Benchmark Model with k = n classes = 2


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


homo	compl	v-meas	ARI	AMI	silhouette
0.015	0.015	0.015	0.020	0.014	0.008

--------------------------
GMM

Plot Model Complexity
cv = diag, k = 2 --> aic = 2209415.225, bic = 2220125.459
cv_type = diag, k = 2 --> homogeneity = 0.000, completeness = 0.000
cv = diag, k = 3 --> aic = 2206237.718, bic = 2222305.746
cv_type = diag, k = 3 --> homogeneity = 0.006, completeness = 0.004
cv = diag, k = 4 --> aic = 2204232.959, bic = 2225658.780
cv_type = diag, k = 4 --> homogeneity = 0.003, completeness = 0.002
cv = diag, k = 5 --> aic = 2202381.375, bic = 2229164.990
cv_type = diag, k = 5 --> homogeneity = 0.007, completeness = 0.003
cv = diag, k = 6 --> aic = 2201613.167, bic = 2233754.576
cv_type = diag, k = 6 --> homogeneity = 0.012, completeness = 0.005
cv = diag, k = 7 --> aic = 2200764.617, bic = 2238263.819
cv_type = diag, k = 7 --> homogeneity = 0.021, completeness = 0.007
cv = diag, k = 8 --> aic = 2200933.652, bic = 2243790.648
cv_type = diag, k = 8 --> homogeneity = 0.032, completenes

## Dimensionality Reduction

In [9]:

from scipy.stats import kurtosis
from sklearn.decomposition import FastICA, KernelPCA, PCA
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


class DimensionalityReduction(ABC):

    def __init__(self, name, n_components, random_seed):
        self.name = name
        self.model = None
        self.n_components = n_components
        self.random_seed = random_seed

    def experiment(self, x_train, x_test, y_train, dataset, perform_model_complexity):
        print('\nTrain on training set')

        if perform_model_complexity:
            self.plot_model_complexity(x_train, dataset)

        x_train_reduced, mse = self.train(x_train)  # fit and reduce training data
        print('Reconstruction error = {:.3f}'.format(mse))
        self.visualize_components(x_train_reduced, y_train, dataset)  # visualize components
        print('Reduced dimension: ', x_train_reduced.shape)
        return x_train_reduced,  self.reduce(x_test)  # reduce test data and return reduced training and test data

    @abstractmethod
    def plot_model_complexity(self, x, dataset, y_train=None):

        pass

    def reconstruct(self, x, x_reduced):

        x_reconstructed = self.model.inverse_transform(x_reduced)  # reconstruct
        mse = np.mean((x - x_reconstructed) ** 2)  # compute MSE
        return mse

    def reduce(self, x):

        return self.model.transform(x)

    def set_n_components(self, n_components):

        self.n_components = n_components
        self.model.set_params(**{'n_components': n_components})

    def train(self, x):

        x_reduced = self.model.fit_transform(x)  # fit model
        mse = self.reconstruct(x, x_reduced)  # reconstruct and compute MSE
        return x_reduced, mse

    def visualize_components(self, x_reduced, y, dataset):

        component1 = '{}1'.format(self.name)  # first component
        component2 = '{}2'.format(self.name)  # second component

        # Create dataframe for visualization
        df = pd.DataFrame(x_reduced[:, :2], columns=[component1, component2])
        df['y'] = y

        # Plot components and save figure
        utils.plot_components(component1, component2, df, self.name)
        utils.save_figure('{}_{}_components'.format(dataset, self.name))


In [10]:
class PrincipalComponents(DimensionalityReduction):

    def __init__(self, n_components=2, random_seed=42):

        super(PrincipalComponents, self).__init__(name='pca', n_components=n_components, random_seed=random_seed)
        self.model = PCA(n_components=n_components, svd_solver='randomized', random_state=random_seed)

    def plot_model_complexity(self, x, dataset):


        print('\nPlot Model Complexity')

        k_range = np.arange(1, x.shape[1]+1)  # range of number of components k to plot over

        # Create PCA object and fit it on training data
        pca = PCA(svd_solver='randomized', random_state=self.random_seed)
        pca.fit(x)

        # Compute the total explained variance given by the chosen number of components
        explained_variance = np.sum(pca.explained_variance_ratio_[:self.n_components])
        print('Explained variance [n components = {}]= {:.3f}'.format(self.n_components, explained_variance))

        # Create subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 5))

        if "MADELON" in dataset:
            ax1.plot(k_range, pca.explained_variance_ratio_, color='green')
            utils.set_axis_title_labels(ax1, title='MADELON - PCA - Eigenvalues distributions',
                                        x_label='Number of components k', y_label='Variance (%)')

            ax2.plot(k_range, np.cumsum(pca.explained_variance_ratio_), color='b')
            utils.set_axis_title_labels(ax2, title='MADELON - PCA - Choosing k with the Variance method',
                                        x_label='Number of components k', y_label='Cumulative Variance (%)')

        else:
            ax1.plot(k_range, pca.explained_variance_ratio_, color='green')
            utils.set_axis_title_labels(ax1, title='MNIST - PCA - Eigenvalues distributions',
                                        x_label='Number of components k', y_label='Variance (%)')

            ax2.plot(k_range, np.cumsum(pca.explained_variance_ratio_), color='b')
            utils.set_axis_title_labels(ax2, title='MNIST - PCA - Choosing k with the Variance method',
                                        x_label='Number of components k', y_label='Cumulative Variance (%)')

        # Save figure
        utils.save_figure_tight('{}_pca_model_complexity'.format(dataset))

In [11]:
class IndependentComponents(DimensionalityReduction):

    def __init__(self, n_components=2, random_seed=42):

        super(IndependentComponents, self).__init__(name='ica', n_components=n_components, random_seed=random_seed)
        self.model = FastICA(n_components=n_components, tol=0.05, max_iter=2000, random_state=random_seed)

    def plot_model_complexity(self, x, dataset):


        print('\nPlot Model Complexity')

        average_kurtosis = []  # list of average kurtosis

        # Define range of number of components k to plot over
        if x.shape[1] < 100:
            k_range = np.arange(1, x.shape[1] + 1)
        else:
            k_range = np.arange(0, int(0.8 * x.shape[1]) + 1, 20)
            k_range[0] = 1

        # For each k in the range
        for k in k_range:

            # Define a new ICA model and fit on training data
            ica = FastICA(n_components=k, tol=0.05, max_iter=20000, random_state=self.random_seed)
            ica.fit(x)

            # Compute kurtosis of components and report the average
            components_kurtosis = kurtosis(ica.components_, axis=1, fisher=False)
            average_kurtosis.append(np.mean(components_kurtosis))
            print('k = {} --> average kurtosis = {:.3f}'.format(k, average_kurtosis[-1]))

        # Create subplots and plot average kurtosis on the first suplot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 5))
        ax1.plot(k_range, average_kurtosis,  '-o', markersize=1, label='Kurtosis')

        # Set title and labels
        utils.set_axis_title_labels(ax1, title='ICA - Choosing k with the Average Kurtosis method',
                                    x_label='Number of components k', y_label='Average Kurtosis')

        # Fit our ICA model on training data
        self.model.fit(x)

        # Define x values to show on plot
        if x.shape[1] < 100:
            x_ticks = np.arange(1, self.n_components + 1)
        else:
            x_ticks = np.arange(0, self.n_components + 1, 50)
            x_ticks[0] = 1

        # Compute kurtosis of components
        components_kurtosis = kurtosis(self.model.components_, axis=1, fisher=False)

        # Plot kurtosis distribution
        ax2.bar(np.arange(1, self.n_components + 1), components_kurtosis, color='g')

        # Set title, labels and x values to show on plot
        ax2.set_xticks(x_ticks)
        utils.set_axis_title_labels(ax2, title='ICA - Components Kurtosis Distribution',
                                    x_label='Independent component', y_label='Kurtosis')

        # Save figure
        utils.save_figure_tight('{}_ica_model_complexity'.format(dataset))


In [12]:
class RandomProjections(DimensionalityReduction):

    def __init__(self, n_components=2, random_runs=100, random_seed=42):

        super(RandomProjections, self).__init__(name='rp', n_components=n_components, random_seed=random_seed)
        self.model = SparseRandomProjection(n_components=n_components, random_state=random_seed)
        self.random_runs = random_runs

    def experiment(self, x_train, x_test, y_train, dataset, perform_model_complexity):


        print('\nTrain on training set')

        if perform_model_complexity:
            self.plot_model_complexity(x_train, dataset)

        # Initialize training errors, reduced training and test data
        train_errors = []
        x_train_reduced = np.zeros((x_train.shape[0], self.n_components))
        x_test_reduced = np.zeros((x_test.shape[0], self.n_components))

        # Perform the defined number of random restarts
        for seed in range(self.random_seed, self.random_seed + self.random_runs):

            # Define new sparse RP model
            self.model = SparseRandomProjection(n_components=self.n_components, random_state=seed)

            # Train on training data
            x_reduced, mse = self.train(x_train)

            # Account for reduced training data and MSE for current run
            x_train_reduced += x_reduced
            train_errors.append(mse)

            # Account for reduced test data
            x_reduced = self.reduce(x_test)
            x_test_reduced += x_reduced

        print('Reconstruction error = {:.3f} with std = {:.3f}'.format(np.mean(train_errors), np.std(train_errors)))

        # Normalize the mean reduced training data and test data
        x_train_reduced /= self.random_runs
        x_test_reduced /= self.random_runs

        # Visualize components
        self.visualize_components(x_train_reduced, y_train, dataset)

        return x_train_reduced, x_test_reduced

    def plot_model_complexity(self, x, dataset):

        print('\nPlot Model Complexity')

        mse_random_runs = []  # list of MSE over random runs

        # Define range of number of components k to plot over
        if x.shape[1] < 100:
            k_range = np.arange(1, x.shape[1] + 1)
        else:
            k_range = np.arange(0, int(0.9 * x.shape[1]) + 1, 20)
            k_range[0] = 1

        # Perform the defined number of random restarts
        for seed in range(self.random_seed, self.random_seed + self.random_runs):

            print('Random run {}'.format(seed + 1 - self.random_seed))
            mse = []  # initialize list of MSE over number of components

            # For each k in the range
            for k in k_range:

                # Define new sparse RP model and reduce training data
                rp = SparseRandomProjection(n_components=k, random_state=np.random.randint(1000))
                x_reduced = rp.fit_transform(x)

                # Compute Pseudo inverse, reconstruct data and compute MSE
                P_inv = np.linalg.pinv(rp.components_.toarray())  # dimensions check : k x m -> m x k
                x_reconstructed = (P_inv @ x_reduced.T).T  # dimensions check: m x k x k x n = m x n -> n x m
                mse.append(np.mean((x - x_reconstructed) ** 2))

            mse_random_runs.append(mse)  # append list of MSE over number of components per current random run

        np.set_printoptions(precision=2)
        print('k = [2, ..., {}] --> \nReconstruction errors = {}'.format(k_range[-1], np.mean(mse_random_runs, axis=0)))

        # Create figure, plot multiple runs and set tile and labels
        plt.figure()
        utils.plot_multiple_random_runs(k_range, mse_random_runs, 'MSE')
        utils.set_plot_title_labels(title='RP - Choosing k with the Reconstruction Error',
                                    x_label='Number of components k', y_label='MSE')

        # Save figure
        utils.save_figure('{}_rp_model_complexity'.format(dataset))

    def reconstruct(self, x, x_reduced):

        # Compute Pseudo inverse, reconstruct data and compute MSE
        P_inv = np.linalg.pinv(self.model.components_.toarray())  # dimensions check : k x m -> m x k
        x_reconstructed = (P_inv @ x_reduced.T).T  # dimensions check: m x k x k x n = m x n -> n x m
        mse = np.mean((x - x_reconstructed) ** 2)
        return mse

In [13]:
class KernelPrincipalComponents(DimensionalityReduction):

    def __init__(self, n_components=2, kernel='rbf', random_seed=42):

        super(KernelPrincipalComponents, self).__init__(name='kpca', n_components=n_components, random_seed=random_seed)
        self.model = KernelPCA(n_components=n_components, kernel=kernel, fit_inverse_transform=True,
                               random_state=random_seed, n_jobs=-1)
        self.kernel = kernel

    def plot_model_complexity(self, x, dataset):

        print('\nPlot Model Complexity')

        k_range = np.arange(1, x.shape[1] + 1)  # range of number of components k to plot over
        if "MADELON" in dataset:
            kernels = ['rbf', 'cosine']  # kernels to plot with
        else:
            kernels = ['rbf', 'poly', 'sigmoid', 'cosine']  # kernels to plot with

        # Create subplots
        fig, ax = plt.subplots(2, 4, figsize=(15, 10))
        ax = ax.ravel()

        # For each kernel
        for i, kernel in enumerate(kernels):

            # Create KPCA object and fit it on training data
            kpca = KernelPCA(n_components=x.shape[1], kernel=kernel, random_state=self.random_seed, n_jobs=-1)
            kpca.fit(x)

            # Compute explained variance ratio from eigenvalues and the total explained variance
            # given by the chosen number of components
            explained_variance_ratio = kpca.lambdas_ / np.sum(kpca.lambdas_)
            explained_variance = np.sum(explained_variance_ratio[:self.n_components])
            print('Kernel = {} - Explained variance [n components = {}]= {:.3f}'.format(kernel,
                                                                                        self.n_components,
                                                                                        explained_variance))

            # Plot histogram of the cumulative explained variance ratio
            ax[2*i].plot(k_range, np.cumsum(explained_variance_ratio), color='blue', label=kernel)

            # Set title, labels and legend
            ax[2*i].legend(loc='best')
            utils.set_axis_title_labels(ax[2*i], title='KPCA - Choosing k with the Variance method',
                                        x_label='Number of components k', y_label='Cumulative Variance (%)')

            # Plot histogram of the explained variance ratio, i.e. the eignevalues distribution
            ax[2*i+1].plot(k_range, explained_variance_ratio, color='green', label=kernel)

            # Set title, labels and legend
            ax[2*i+1].legend(loc='best')
            utils.set_axis_title_labels(ax[2*i+1], title='KPCA - Eigenvalues distributions',
                                        x_label='Number of components k', y_label='Variance (%)')

        # Save figure
        utils.save_figure_tight('{}_kpca_model_complexity'.format(dataset))

In [14]:
class LinearDiscriminantAnalysis(DimensionalityReduction):
    def __init__(self, n_components=10, random_seed=42):

        super(LinearDiscriminantAnalysis, self).__init__(name='lda', n_components=n_components, random_seed=random_seed)
        self.model = LDA(n_components=n_components)

    def experiment(self, x_train, x_test, y_train, dataset, perform_model_complexity):
        print('\nTrain on training set')

        if perform_model_complexity:
            self.plot_model_complexity(x_train, dataset, y_train)

        x_reduced = self.model.fit_transform(x_train, y_train)  # fit model
        mse = self.reconstruct(x_train, x_reduced)  # reconstruct and compute MSE
        # fit and reduce training data
        print('Reconstruction error = {:.3f}'.format(mse))
        # self.visualize_components(x_train_reduced, y_train, dataset)  # visualize components
        print('Reduced dimension: ', x_reduced.shape)


    def plot_model_complexity(self, x, dataset, y=None):

        print('\nPlot Model Complexity')

        explained_variance = []  # list of average kurtosis

        # Define range of number of components k to plot over
        if x.shape[1] < 100:
            k_range = np.arange(1, x.shape[1] + 1)
        else:
            k_range = np.arange(0, int(0.8 * x.shape[1]) + 1, 20)
            k_range[0] = 1

        for n in range(1,  10):

            lda = LDA(n_components=n, tol=0.01)
            lda.fit(x, y)
            print(lda.explained_variance_ratio_)
        # Compute the total explained variance given by the chosen number of components
        explained_variance = np.sum(lda.explained_variance_ratio_[:self.n_components])
        print('Explained variance [n components = {}]= {:.3f}'.format(self.n_components, explained_variance))

        # Create subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 5))

        if "MADELON" in dataset:
            ax1.plot(k_range, lda.explained_variance_ratio_, color='green')
            utils.set_axis_title_labels(ax1, title='MADELON - PCA - Eigenvalues distributions',
                                        x_label='Number of components k', y_label='Variance (%)')

            ax2.plot(k_range, np.cumsum(lda.explained_variance_ratio_), color='b')
            utils.set_axis_title_labels(ax2, title='MADELON - PCA - Choosing k with the Variance method',
                                        x_label='Number of components k', y_label='Cumulative Variance (%)')

        else:
            ax1.plot(k_range, lda.explained_variance_ratio_, color='green')
            utils.set_axis_title_labels(ax1, title='MNIST - PCA - Eigenvalues distributions',
                                        x_label='Number of components k', y_label='Variance (%)')

            ax2.plot(k_range, np.cumsum(lda.explained_variance_ratio_), color='b')
            utils.set_axis_title_labels(ax2, title='MNIST - PCA - Choosing k with the Variance method',
                                        x_label='Number of components k', y_label='Cumulative Variance (%)')

        # Save figure
        utils.save_figure_tight('{}_pca_model_complexity'.format(dataset))

## DR experiments


In [15]:
def dimensionality_reduction(x_train, x_test, y_train, **kwargs):
    print('\n--------------------------')
    print('PCA')
    print('--------------------------')

    # Declare PCA, perform experiments by reducing the dataset and perform clustering experiments on it
    pca = PrincipalComponents(n_components=kwargs['pca_n_components'])
    x_pca = pca.experiment(x_train, x_test, y_train,
                           dataset=kwargs['dataset'],
                           perform_model_complexity=kwargs['perform_model_complexity'])

    clustering(x_pca[0],  x_pca[1], y_train,
               dataset=kwargs['dataset'] + '_pca_reduced',
               kmeans_n_clusters=kwargs['pca_kmeans_n_clusters'],
               em_n_clusters=kwargs['pca_em_n_clusters'], em_covariance=kwargs['pca_em_covariance'],
               perform_model_complexity=kwargs['perform_model_complexity'])

    print('\n--------------------------')
    print('ICA')
    print('--------------------------')

    # # Declare ICA, perform experiments by reducing the dataset and perform clustering experiments on it
    ica = IndependentComponents(n_components=kwargs['ica_n_components'])
    x_ica = ica.experiment(x_train, x_test, y_train,
                           dataset=kwargs['dataset'],
                           perform_model_complexity=kwargs['perform_model_complexity'])

    clustering(x_ica[0],  x_ica[1], y_train,
               dataset=kwargs['dataset'] + '_ica_reduced',
               kmeans_n_clusters=kwargs['ica_kmeans_n_clusters'],
               em_n_clusters=kwargs['ica_em_n_clusters'], em_covariance=kwargs['ica_em_covariance'],
               perform_model_complexity=kwargs['perform_model_complexity'])


    print('\n--------------------------')
    print('RP')
    print('--------------------------')

    # # Declare RP, perform experiments by reducing the dataset and perform clustering experiments on it
    rp = RandomProjections(n_components=kwargs['rp_n_components'])
    x_rp = rp.experiment(x_train, x_test, y_train,
                         dataset=kwargs['dataset'],
                         perform_model_complexity=kwargs['perform_model_complexity'])

    clustering(x_rp[0], x_rp[1], y_train,
               dataset=kwargs['dataset'] + '_rp_reduced',
               kmeans_n_clusters=kwargs['rp_kmeans_n_clusters'],
               em_n_clusters=kwargs['rp_em_n_clusters'], em_covariance=kwargs['rp_em_covariance'],
               perform_model_complexity=kwargs['perform_model_complexity'])

    print('\n--------------------------')
    print('KPCA')
    print('--------------------------')

    # Declare KPCA, perform experiments by reducing the dataset and perform clustering experiments on it
    kpca = KernelPrincipalComponents(n_components=kwargs['kpca_n_components'], kernel=kwargs['kpca_kernel'])
    x_kpca = kpca.experiment(x_train, x_test, y_train,
                             dataset=kwargs['dataset'],
                             perform_model_complexity=kwargs['perform_model_complexity'])

    clustering(x_kpca[0], x_kpca[1], y_train,
               dataset=kwargs['dataset'] + '_kpca_reduced',
               kmeans_n_clusters=kwargs['kpca_kmeans_n_clusters'],
               em_n_clusters=kwargs['kpca_em_n_clusters'], em_covariance=kwargs['kpca_em_covariance'],
               perform_model_complexity=kwargs['perform_model_complexity'])

    print('\n--------------------------')
    print('LDA')
    print('--------------------------')

    # # Declare RP, perform experiments by reducing the dataset and perform clustering experiments on it
    # lda = LinearDiscriminantAnalysis(n_components=kwargs['rp_n_components'])
    # x_lda = lda.experiment(x_train, x_test, y_train,
    #                      dataset=kwargs['dataset'],
    #                      perform_model_complexity=kwargs['perform_model_complexity'])
    #
    # clustering(x_lda[0], x_lda[1], y_train,
    #            dataset=kwargs['dataset'] + '_rp_reduced',
    #            kmeans_n_clusters=kwargs['lda_kmeans_n_clusters'],
    #            em_n_clusters=kwargs['lda_em_n_clusters'], em_covariance=kwargs['lda_em_covariance'],
    #            perform_model_complexity=kwargs['perform_model_complexity'])

    return x_pca, x_ica, x_kpca, x_rp


In [16]:
dataset = 'MNIST'
perform_model_complexity = True
x_pca, x_ica, x_kpca, x_rp = dimensionality_reduction(x_train, x_test, y_train,
                                                              dataset=dataset,
                                                              pca_n_components=260, pca_kmeans_n_clusters=2,
                                                              pca_em_n_clusters=6, pca_em_covariance='full',
                                                              ica_n_components=320, ica_kmeans_n_clusters=2,
                                                              ica_em_n_clusters=10, ica_em_covariance='diag',
                                                              kpca_n_components=260, kpca_kernel='cosine',
                                                              kpca_kmeans_n_clusters=2,
                                                              kpca_em_n_clusters=3, kpca_em_covariance='full',
                                                              rp_n_components=500, rp_kmeans_n_clusters=2,
                                                              rp_em_n_clusters=2, rp_em_covariance='tied',
                                                              lda_n_components=10, lda_kmeans_n_clusters=2,
                                                              lda_em_n_clusters=5, lda_em_covariance='diag',
                                                              perform_model_complexity=perform_model_complexity)



--------------------------
PCA
--------------------------

Train on training set

Plot Model Complexity
Explained variance [n components = 260]= 0.947
Reconstruction error = 0.045
Reduced dimension:  (5600, 260)

--------------------------
kMeans

Plot Model Complexity


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 1 -->  inertia = 3526216.864  silhouette = 0.000 homogeneity = 0.000 completeness = 1.000


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 2 -->  inertia = 3373636.537  silhouette = 0.087 homogeneity = 0.130 completeness = 0.323


...

k = 12 -->  inertia = 2833702.693  silhouette = 0.459 homogeneity = 0.014 completeness = 0.459


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 15 -->  inertia = 2755281.731  silhouette = 0.497 homogeneity = 0.022 completeness = 0.457


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


KeyboardInterrupt: 

In [17]:
dataset = 'MADELON'
perform_model_complexity = True
x_pca, x_ica, x_kpca, x_rp = dimensionality_reduction(x_train, x_test, y_train,
                                                              dataset=dataset,
                                                              pca_n_components=260, pca_kmeans_n_clusters=2,
                                                              pca_em_n_clusters=6, pca_em_covariance='full',
                                                              ica_n_components=320, ica_kmeans_n_clusters=2,
                                                              ica_em_n_clusters=10, ica_em_covariance='diag',
                                                              kpca_n_components=260, kpca_kernel='cosine',
                                                              kpca_kmeans_n_clusters=2,
                                                              kpca_em_n_clusters=3, kpca_em_covariance='full',
                                                              rp_n_components=500, rp_kmeans_n_clusters=2,
                                                              rp_em_n_clusters=2, rp_em_covariance='tied',
                                                              lda_n_components=10, lda_kmeans_n_clusters=2,
                                                              lda_em_n_clusters=5, lda_em_covariance='diag',
                                                              perform_model_complexity=perform_model_complexity)




--------------------------
PCA
--------------------------

Train on training set

Plot Model Complexity
Explained variance [n components = 260]= 0.947
Reconstruction error = 0.045
Reduced dimension:  (5600, 260)

--------------------------
kMeans

Plot Model Complexity


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 2 -->  inertia = 3373636.537  silhouette = 0.087 homogeneity = 0.130 completeness = 0.323


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


k = 3 -->  inertia = 3276418.870  silhouette = 0.222 homogeneity = 0.055 completeness = 0.516


...

k = 10 -->  inertia = 4014.340  silhouette = 0.477 homogeneity = 0.072 completeness = 0.484

Train on training set with k=2


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


homo	compl	v-meas	ARI	AMI	silhouette
0.080	0.319	0.128	0.049	0.128	0.058

Benchmark Model with k = n classes = 10


/home/ryanym/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


homo	compl	v-meas	ARI	AMI	silhouette
0.477	0.484	0.480	0.350	0.479	0.072

--------------------------
GMM

Plot Model Complexity
cv = diag, k = 2 --> aic = -5574459.158, bic = -5567556.785
cv_type = diag, k = 2 --> homogeneity = 0.023, completeness = 0.099
cv = diag, k = 3 --> aic = -5615765.926, bic = -5605409.051
cv_type = diag, k = 3 --> homogeneity = 0.064, completeness = 0.158
cv = diag, k = 4 --> aic = -5638399.889, bic = -5624588.512
cv_type = diag, k = 4 --> homogeneity = 0.125, completeness = 0.254
cv = diag, k = 5 --> aic = -5649311.319, bic = -5632045.440
cv_type = diag, k = 5 --> homogeneity = 0.130, completeness = 0.231
cv = diag, k = 6 --> aic = -5656965.126, bic = -5636244.745
cv_type = diag, k = 6 --> homogeneity = 0.172, completeness = 0.270
cv = diag, k = 7 --> aic = -5662788.587, bic = -5638613.704
cv_type = diag, k = 7 --> homogeneity = 0.175, completeness = 0.243
cv = diag, k = 8 --> aic = -5669060.904, bic = -5641431.519
cv_type = diag, k = 8 --> homogeneity = 0.19

## NN with DR

In [18]:
# Neural Network class

import time

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier


class NeuralNetwork:

    def __init__(self, layer1_nodes, layer2_nodes, learning_rate):
        self.model = MLPClassifier(hidden_layer_sizes=(layer1_nodes), activation='relu',
                                   solver='sgd', alpha=0.01, batch_size=200, learning_rate='constant',
                                   learning_rate_init=learning_rate, max_iter=1000, tol=1e-4,
                                   early_stopping=False, validation_fraction=0.1, momentum=0.5,
                                   n_iter_no_change=100, random_state=42)

    def evaluate(self, x_test, y_test):

        predictions = self.predict(x_test)  # predict on test data

        print('\nEvaluate on the Test Set')
        print(classification_report(y_test, predictions))  # produce classification report
        print('Confusion Matrix:')
        print(confusion_matrix(y_test, predictions))  # produce confusion matrix

    def fit(self, x_train, y_train):

        # Fit the model and report training time
        start_time = time.time()
        self.model.fit(x_train, y_train)
        end_time = time.time()

        print('\nFitting Training Set: {:.4f} seconds'.format(end_time-start_time))

    def predict(self, x):

        # Predict and report inference time
        start_time = time.time()
        predictions = self.model.predict(x)
        end_time = time.time()

        print('\nPredicting on Testing Set: {:.4f} seconds'.format(end_time-start_time))

        return predictions

    def experiment(self, x_train, x_test, y_train, y_test):
        self.fit(x_train, y_train)
        self.evaluate(x_test, y_test)



## NN experiments

In [19]:
def neural_network(x_train, x_test, y_train, y_test,
                   x_pca, x_ica, x_kpca, x_rp,
                   x_kmeans, x_gmm, **kwargs):
    """Perform neural network experiment.

        Args:
           x_train (ndarray): training data.
           x_test (ndarray): test data.
           y_train (ndarray): training labels.
           y_test (ndarray): test labels.
           x_pca (ndarray): reduced dataset by PCA.
           x_ica (ndarray): reduced dataset by ICA.
           x_kpca (ndarray): reduced dataset by KPCA.
           x_rp (ndarray): reduced dataset by RP.
           x_kmeans (ndarray): clusters produced by k-Means.
           x_gmm (ndarray): clusters produced by Gaussian Mixture Models.
           kwargs (dict): additional arguments to pass:
                    - layer1_nodes (int): number of neurons in first layer.
                    - layer2_nodes (int): number of neurons in second layer.
                    - learning_rate (float): learning rate.

        Returns:
           None.
        """

    print('\n--------------------------')
    print('NN')
    print('--------------------------')

    # Declare Neural Network and perform experiments on the original dataset
    nn = NeuralNetwork(layer1_nodes=kwargs['layer1_nodes'],
                       layer2_nodes=kwargs['layer2_nodes'],
                       learning_rate=kwargs['learning_rate'])
    nn.experiment(x_train, x_test, y_train, y_test)

    print('\n--------------------------')
    print('PCA + NN')
    print('--------------------------')

    # Declare Neural Network and perform experiments on the reduced dataset by PCA
    nn = NeuralNetwork(layer1_nodes=kwargs['layer1_nodes'],
                       layer2_nodes=kwargs['layer2_nodes'],
                       learning_rate=kwargs['learning_rate'])
    nn.experiment(x_pca[0], x_pca[1], y_train, y_test)

    print('\n--------------------------')
    print('ICA + NN')
    print('--------------------------')

    # Declare Neural Network and perform experiments on the reduced dataset by ICA
    nn = NeuralNetwork(layer1_nodes=kwargs['layer1_nodes'],
                       layer2_nodes=kwargs['layer2_nodes'],
                       learning_rate=kwargs['learning_rate'])
    nn.experiment(x_ica[0], x_ica[1], y_train, y_test)

    print('\n--------------------------')
    print('KPCA + NN')
    print('--------------------------')

    # Declare Neural Network and perform experiments on the reduced dataset by KPCA
    nn = NeuralNetwork(layer1_nodes=kwargs['layer1_nodes'],
                       layer2_nodes=kwargs['layer2_nodes'],
                       learning_rate=kwargs['learning_rate'])
    nn.experiment(x_kpca[0], x_kpca[1], y_train, y_test)

    print('\n--------------------------')
    print('RP+ NN')
    print('--------------------------')

    # Declare Neural Network and perform experiments on the reduced dataset by RP
    nn = NeuralNetwork(layer1_nodes=kwargs['layer1_nodes'],
                       layer2_nodes=kwargs['layer2_nodes'],
                       learning_rate=kwargs['learning_rate'])
    nn.experiment(x_rp[0], x_rp[1], y_train, y_test)

    print('\n--------------------------')
    print('KMEANS+ NN')
    print('--------------------------')

    # Declare Neural Network
    nn = NeuralNetwork(layer1_nodes=kwargs['layer1_nodes'],
                       layer2_nodes=kwargs['layer2_nodes'],
                       learning_rate=kwargs['learning_rate'])

    # Augment the original dataset by adding clusters produced by k-Means as features
    x_kmeans_normalized = (x_kmeans[0] - np.mean(x_kmeans[0])) / np.std(x_kmeans[0])
    x_kmeans_normalized = np.expand_dims(x_kmeans_normalized, axis=1)
    x_train_new = np.append(x_train, x_kmeans_normalized, axis=1)
    x_kmeans_normalized = (x_kmeans[1] - np.mean(x_kmeans[1])) / np.std(x_kmeans[1])
    x_kmeans_normalized = np.expand_dims(x_kmeans_normalized, axis=1)
    x_test_new = np.append(x_test, x_kmeans_normalized, axis=1)

    # Perform experiments on it
    nn.experiment(x_train_new, x_test_new, y_train, y_test)

    print('\n--------------------------')
    print('GMM+ NN')
    print('--------------------------')

    # Declare Neural Network
    nn = NeuralNetwork(layer1_nodes=kwargs['layer1_nodes'],
                       layer2_nodes=kwargs['layer2_nodes'],
                       learning_rate=kwargs['learning_rate'])

    # Augment the original dataset by adding clusters produced by Gaussian Mixture Models as features
    x_gmm_normalized = (x_gmm[0] - np.mean(x_gmm[0])) / np.std(x_gmm[0])
    x_gmm_normalized = np.expand_dims(x_gmm_normalized, axis=1)
    x_train_new = np.append(x_train, x_gmm_normalized, axis=1)
    x_gmm_normalized = (x_gmm[1] - np.mean(x_gmm[1])) / np.std(x_gmm[1])
    x_gmm_normalized = np.expand_dims(x_gmm_normalized, axis=1)
    x_test_new = np.append(x_test, x_gmm_normalized, axis=1)

    # Perform experiments on it
    nn.experiment(x_train_new, x_test_new, y_train, y_test)

In [20]:
 neural_network(x_train, x_test, y_train, y_test,
                       x_pca, x_ica, x_kpca, x_rp,
                       kmeans_clusters, gmm_clusters,
                       layer1_nodes=150, layer2_nodes=100, learning_rate=0.06)



--------------------------
NN
--------------------------

Fitting Training Set: 18.9585 seconds

Predicting on Testing Set: 0.0061 seconds

Evaluate on the Test Set
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       138
           1       0.93      0.98      0.96       157
           2       0.94      0.86      0.90       140
           3       0.91      0.88      0.89       143
           4       0.93      0.94      0.93       136
           5       0.94      0.92      0.93       126
           6       0.92      0.96      0.94       138
           7       0.92      0.92      0.92       146
           8       0.90      0.93      0.91       137
           9       0.91      0.90      0.91       139

    accuracy                           0.93      1400
   macro avg       0.93      0.93      0.93      1400
weighted avg       0.93      0.93      0.93      1400

Confusion Matrix:
[[136   0   1   0   0   0   1   0   0   0]
 [  0 154   0 

/home/ryanym/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(



Fitting Training Set: 72.8614 seconds

Predicting on Testing Set: 0.0032 seconds

Evaluate on the Test Set
              precision    recall  f1-score   support

           0       0.94      0.99      0.96       138
           1       0.89      0.98      0.93       157
           2       0.91      0.83      0.87       140
           3       0.90      0.84      0.87       143
           4       0.86      0.91      0.89       136
           5       0.83      0.87      0.84       126
           6       0.89      0.91      0.90       138
           7       0.95      0.87      0.91       146
           8       0.89      0.85      0.87       137
           9       0.87      0.88      0.87       139

    accuracy                           0.89      1400
   macro avg       0.89      0.89      0.89      1400
weighted avg       0.89      0.89      0.89      1400

Confusion Matrix:
[[136   0   0   0   0   0   1   0   0   1]
 [  0 154   0   0   1   0   0   0   2   0]
 [  1   2 116   5   3   1   4

/home/ryanym/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(



Fitting Training Set: 71.2502 seconds

Predicting on Testing Set: 0.0032 seconds

Evaluate on the Test Set
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       138
           1       0.96      0.97      0.97       157
           2       0.91      0.88      0.89       140
           3       0.91      0.88      0.90       143
           4       0.91      0.92      0.92       136
           5       0.90      0.90      0.90       126
           6       0.92      0.94      0.93       138
           7       0.94      0.92      0.93       146
           8       0.88      0.93      0.90       137
           9       0.91      0.89      0.90       139

    accuracy                           0.92      1400
   macro avg       0.92      0.92      0.92      1400
weighted avg       0.92      0.92      0.92      1400

Confusion Matrix:
[[135   0   1   0   1   0   1   0   0   0]
 [  0 153   0   1   0   0   0   1   2   0]
 [  0   0 123   2   2   1   2